In [1]:
import random
from copy import deepcopy as dc
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

CSV_PATH = "datasets/final/datasets_lagwise/PM_Combined_AQI_2022_2024.csv"

BATCH_SIZE = 32
LR = 1e-3
EPOCHS = 200
PATIENCE = 20
FEATURE_RANGE = (-1, 1)

lag_list = [1, 7, 14, 30]
lambda_configs = [
    (0.0, 1.0),
    (0.3, 0.7),
    (0.5, 0.5),
    (0.7, 0.3),
    (1.0, 0.0),
]

df = pd.read_csv(CSV_PATH)
df["DATE"] = pd.to_datetime(df["Date"], format="%d-%m-%y", dayfirst=True, errors="coerce")
df = df.dropna(subset=["DATE", "Daily_Mean_PM", "Daily_AQI_Value"]).reset_index(drop=True)

data = df[["DATE", "Daily_Mean_PM", "Daily_AQI_Value"]].copy()
data = data.sort_values("DATE").reset_index(drop=True)

print("\n--- Base Data Frame (Head) ---")
print(data.head())
print("-" * 50)
print("Shape:", data.shape)


def build_supervised_table(df_in: pd.DataFrame, LAG: int) -> pd.DataFrame:
    lag_col = f"AQI_Targeted_Value_LAG_{LAG}"
    pm_lag_col = f"PM_Targeted_Value_LAG_{LAG}"
    df_local = dc(df_in[["DATE", "Daily_Mean_PM", "Daily_AQI_Value", lag_col, pm_lag_col]]).set_index("DATE")
    cols = ["Daily_Mean_PM", "Daily_AQI_Value", lag_col, pm_lag_col]
    return df_local[cols].dropna()


def epa_aqi_from_pm25(pm25_real: torch.Tensor) -> torch.Tensor:
    breakpoints = [
        ((0.0, 9.0), (0, 50)),
        ((9.001, 35.4), (51, 100)),
        ((35.401, 55.4), (101, 150)),
        ((55.401, 125.4), (151, 200)),
        ((125.401, 225.4), (201, 300)),
        ((225.401, 325.4), (301, 400)),
        ((325.401, 500.4), (401, 500)),
    ]
    aqi = torch.zeros_like(pm25_real)
    for (bp_lo, bp_hi), (i_lo, i_hi) in breakpoints:
        mask = (pm25_real >= bp_lo) & (pm25_real <= bp_hi)
        aqi[mask] = ((i_hi - i_lo) / (bp_hi - bp_lo)) * (pm25_real[mask] - bp_lo) + i_lo
    aqi[pm25_real > 500.4] = 500.0
    return aqi


def scale_as_first_column(col_real: torch.Tensor, full_scaler: MinMaxScaler) -> torch.Tensor:
    col_np = col_real.detach().cpu().numpy()
    B = col_np.shape[0]
    dummy = np.zeros((B, full_scaler.n_features_in_), dtype=col_np.dtype)
    dummy[:, 0] = col_np[:, 0]
    scaled = full_scaler.transform(dummy)[:, [0]]
    return torch.tensor(scaled, dtype=col_real.dtype, device=col_real.device)


def inverse_first_column(series_scaled_1d: np.ndarray, n_features_total: int, full_scaler: MinMaxScaler) -> np.ndarray:
    d = np.zeros((series_scaled_1d.shape[0], n_features_total))
    d[:, 0] = series_scaled_1d
    return full_scaler.inverse_transform(d)[:, 0]


class SeqDataset(Dataset):
    def __init__(self, X, y, pm_t_real):
        self.X, self.y, self.pm = X, y, pm_t_real

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.pm[idx]


class LSTMRegressorDeepCustom(nn.Module):
    def __init__(
        self,
        input_size: int = 2,
        lstm_hidden_size_1: int = 32,
        lstm_hidden_size_2: int = 32,
        bidirectional: bool = False,
        mlp_hidden_sizes: tuple = (128, 64, 32),
        mlp_dropout: float = 0.1,
    ):
        super().__init__()
        self.bidirectional = bidirectional

        self.lstm1 = nn.LSTM(
            input_size=input_size,
            hidden_size=lstm_hidden_size_1,
            num_layers=1,
            batch_first=True,
            bidirectional=bidirectional,
        )

        lstm2_input_size = lstm_hidden_size_1 * (2 if bidirectional else 1)
        self.lstm2 = nn.LSTM(
            input_size=lstm2_input_size,
            hidden_size=lstm_hidden_size_2,
            num_layers=1,
            batch_first=True,
            bidirectional=bidirectional,
        )

        lstm_out_dim = lstm_hidden_size_2 * (2 if bidirectional else 1)

        layers = []
        prev_dim = lstm_out_dim
        for h in mlp_hidden_sizes:
            layers += [
                nn.Linear(prev_dim, h),
                nn.ReLU(inplace=True),
                nn.Dropout(p=mlp_dropout),
            ]
            prev_dim = h
        layers += [nn.Linear(prev_dim, 1)]
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = out[:, -1, :]
        out = self.mlp(out)
        return out


criterion = nn.MSELoss()


def compute_data_loss(model, xb, yb):
    preds = model(xb)
    return criterion(preds, yb)


def compute_physics_loss_t(model, xb, pm_t_real, scaler):
    preds_scaled = model(xb)
    aqi_phys_real = epa_aqi_from_pm25(pm_t_real)
    aqi_phys_scaled = scale_as_first_column(aqi_phys_real, scaler)
    return criterion(preds_scaled, aqi_phys_scaled)


def train_and_validate(
    model,
    train_loader,
    val_loader,
    scaler,
    epochs=EPOCHS,
    lr=LR,
    patience=PATIENCE,
    lambda_data=0.7,
    lambda_phys=0.3,
):
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode="min", factor=0.5, patience=5, min_lr=1e-3)

    best_val = float("inf")
    best_state = None
    no_improve = 0
    tr_hist, va_hist = [], []

    for ep in range(1, epochs + 1):
        model.train()
        tr_total = tr_data = tr_phys = 0.0
        for xb, yb, pm in train_loader:
            xb, yb, pm = xb.to(device), yb.to(device), pm.to(device)
            optim.zero_grad()
            loss_d = compute_data_loss(model, xb, yb)
            loss_p = compute_physics_loss_t(model, xb, pm, scaler)
            loss = lambda_data * loss_d + lambda_phys * loss_p
            loss.backward()
            optim.step()
            tr_total += loss.item()
            tr_data += loss_d.item()
            tr_phys += loss_p.item()

        tr_total /= max(1, len(train_loader))
        tr_data /= max(1, len(train_loader))
        tr_phys /= max(1, len(train_loader))
        tr_hist.append(tr_total)

        model.eval()
        va_total = va_data = va_phys = 0.0
        with torch.no_grad():
            for xb, yb, pm in val_loader:
                xb, yb, pm = xb.to(device), yb.to(device), pm.to(device)
                preds = model(xb)
                loss_d = criterion(preds, yb)
                loss_p = compute_physics_loss_t(model, xb, pm, scaler)
                loss = lambda_data * loss_d + lambda_phys * loss_p
                va_total += loss.item()
                va_data += loss_d.item()
                va_phys += loss_p.item()

        va_total /= max(1, len(val_loader))
        va_data /= max(1, len(val_loader))
        va_phys /= max(1, len(val_loader))
        va_hist.append(va_total)

        sched.step(va_total)

        print(
            f"Epoch {ep:03d} | "
            f"Train {tr_total:.6f} (data {tr_data:.6f}, phys {tr_phys:.6f}) | "
            f"Val {va_total:.6f} (data {va_data:.6f}, phys {va_phys:.6f})"
        )

        if va_total < best_val - 1e-6:
            best_val = va_total
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {ep} (best val = {best_val:.6f})")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return tr_hist, va_hist


results = {
    "LAG": [],
    "Model": [],
    "lambda_data": [],
    "lambda_phys": [],
    "MAE": [],
    "RMSE": [],
    "NMSE": [],
}


for LAG in lag_list:
    print("\n" + "=" * 60)
    print(f"Preparing and training LSTM+Physics for LAG = {LAG}")
    print("=" * 60)

    tmp = data.copy()
    tmp[f"AQI_Targeted_Value_LAG_{LAG}"] = tmp["Daily_AQI_Value"].shift(-LAG)
    tmp[f"PM_Targeted_Value_LAG_{LAG}"] = tmp["Daily_Mean_PM"].shift(-LAG)

    supervised = build_supervised_table(tmp, LAG)
    print("\n--- Supervised Data (Head) ---")
    print(supervised.head())
    print("Shape:", supervised.shape)
    print("-" * 50)

    arr = supervised.to_numpy()
    y_real = arr[:, 2:3]
    pm_t = arr[:, 3:4]
    X_feats = arr[:, 0:2]

    N = arr.shape[0]
    split_idx = int(N * 0.80)

    tr_y, te_y = y_real[:split_idx], y_real[split_idx:]
    tr_pm, te_pm = pm_t[:split_idx], pm_t[split_idx:]
    tr_X, te_X = X_feats[:split_idx], X_feats[split_idx:]

    train_stack = np.concatenate([tr_y, tr_X], axis=1)
    test_stack = np.concatenate([te_y, te_X], axis=1)

    full_scaler = MinMaxScaler(feature_range=FEATURE_RANGE)
    train_scaled = full_scaler.fit_transform(train_stack)
    test_scaled = full_scaler.transform(test_stack)

    feature_dim = train_scaled.shape[1] - 1
    X_train = torch.tensor(train_scaled[:, 1:].reshape(-1, 1, feature_dim), dtype=torch.float32)
    y_train = torch.tensor(train_scaled[:, :1], dtype=torch.float32)
    X_test = torch.tensor(test_scaled[:, 1:].reshape(-1, 1, feature_dim), dtype=torch.float32)
    y_test = torch.tensor(test_scaled[:, :1], dtype=torch.float32)

    pm_t_train_real = torch.tensor(tr_pm, dtype=torch.float32)
    pm_t_test_real = torch.tensor(te_pm, dtype=torch.float32)

    train_loader = DataLoader(SeqDataset(X_train, y_train, pm_t_train_real), batch_size=BATCH_SIZE, shuffle=False)
    val_loader = DataLoader(SeqDataset(X_test, y_test, pm_t_test_real), batch_size=BATCH_SIZE, shuffle=False)

    tr_y_tensor = torch.tensor(tr_y, dtype=torch.float32).to(device)
    with torch.no_grad():
        aqi_phys_train = epa_aqi_from_pm25(pm_t_train_real.to(device))
        abs_diff = torch.abs(aqi_phys_train - tr_y_tensor)
        print("\n--- First 10 Samples: True vs Physics AQI (Target Day) ---")
        for i in range(min(10, len(tr_y_tensor))):
            true_val = float(tr_y_tensor[i].cpu().item())
            phys_val = float(aqi_phys_train[i].cpu().item())
            diff_val = float(abs_diff[i].cpu().item())
            print(f"Sample {i+1:02d}: True={true_val:.3f}, Physics={phys_val:.3f}, Diff={diff_val:.3f}")

    for lambda_data, lambda_phys in lambda_configs:
        print("\n" + "-" * 60)
        print(
            f"Training LSTM+Physics | LAG = {LAG}, "
            f"lambda_data = {lambda_data}, lambda_phys = {lambda_phys}"
        )
        print("-" * 60)

        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)

        model = LSTMRegressorDeepCustom(
            input_size=feature_dim,
            lstm_hidden_size_1=32,
            lstm_hidden_size_2=32,
            bidirectional=False,
            mlp_hidden_sizes=(128, 64, 32),
            mlp_dropout=0.1,
        ).to(device)

        train_losses, val_losses = train_and_validate(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            scaler=full_scaler,
            epochs=EPOCHS,
            lr=LR,
            patience=PATIENCE,
            lambda_data=lambda_data,
            lambda_phys=lambda_phys,
        )

        model.eval()
        with torch.no_grad():
            te_pred_s = model(X_test.to(device)).cpu().numpy().ravel()

        n_features_total = full_scaler.n_features_in_
        te_pred = inverse_first_column(te_pred_s, n_features_total, full_scaler)
        y_te = inverse_first_column(y_test.cpu().numpy().ravel(), n_features_total, full_scaler)

        mae_te = mean_absolute_error(y_te, te_pred)
        rmse_te = np.sqrt(mean_squared_error(y_te, te_pred))

        mse_te = mean_squared_error(y_te, te_pred)
        denom = np.mean((y_te - np.mean(y_te)) ** 2)
        eps = 1e-12
        nmse_te = mse_te / (denom + eps)

        print("\n--- Test Evaluation (Real AQI Units) ---")
        print(f"LAG = {LAG}, λ_data = {lambda_data}, λ_phys = {lambda_phys}")
        print(f"MAE:  {mae_te:.4f}")
        print(f"RMSE: {rmse_te:.4f}")
        print(f"NMSE: {nmse_te:.4f}")

        results["LAG"].append(LAG)
        results["Model"].append("LSTM+Physics")
        results["lambda_data"].append(lambda_data)
        results["lambda_phys"].append(lambda_phys)
        results["MAE"].append(mae_te)
        results["RMSE"].append(rmse_te)
        results["NMSE"].append(nmse_te)


results_df = pd.DataFrame(results)
results_df.to_csv("PM25_LSTM_Physics_LambdaSweep_Results.csv", index=False)

print("\n===== Final LSTM+Physics Lambda Sweep Results =====")
print(results_df)


Using device: cpu

--- Base Data Frame (Head) ---
        DATE  Daily_Mean_PM  Daily_AQI_Value
0 2022-01-01            6.1        34.000000
1 2022-01-02            4.6        26.000000
2 2022-01-03            9.6        52.000000
3 2022-01-04            5.3        29.000000
4 2022-01-05            5.9        32.666667
--------------------------------------------------
Shape: (1094, 3)

Preparing and training LSTM+Physics for LAG = 1

--- Supervised Data (Head) ---
            Daily_Mean_PM  Daily_AQI_Value  AQI_Targeted_Value_LAG_1  \
DATE                                                                   
2022-01-01            6.1        34.000000                 26.000000   
2022-01-02            4.6        26.000000                 52.000000   
2022-01-03            9.6        52.000000                 29.000000   
2022-01-04            5.3        29.000000                 32.666667   
2022-01-05            5.9        32.666667                 39.000000   

            PM_Targeted_Va